In [2]:
import cv2
import numpy as np
import socket
import struct
import time

# Define the host and port for the server
HOST = ''
PORT = 5000
accumulator = 0
tour = 0

# Create a buffer to store image data
buffer = bytearray()

# Create a socket and bind it to the host and port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    print(f"Waiting for a connection on {HOST}:{PORT}")

    # Accept a connection from a client
    conn, addr = s.accept()
    print(f"Connected to {addr}")

    # Loop to receive images from the client
    while True:
        tour += 1
        print("Tour : " + str(tour))
        # Receive the size of the package
        package_size_data = conn.recv(4)
        if not package_size_data:
            print("Connection closed by client")
            break
        package_size = struct.unpack("I", package_size_data)[0]
        print(f"Expected package size: {package_size}")

        # Receive the size of the entire image
        image_size_data = conn.recv(4)
        if not image_size_data:
            break
        image_size = struct.unpack("I", image_size_data)[0]
        print(f"Expected image size: {image_size}")
        accumulator += package_size
        print("Remaining Size :" + str(image_size - accumulator))

        # Receive the image data
        data = conn.recv(package_size)
        if not data:
            break

        # Append received data to buffer
        buffer.extend(data)
        
        print("Buffer : " + str(len(buffer)))
        
        print("xxxxxxxxxxxxxxxx")

        # Check if we've received the entire image
        if len(buffer) >= image_size:
            # Decode the image data and display the image
            nparr = np.frombuffer(buffer, np.uint8)
            
            try:
                img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                cv2.imshow('Received Image', img)
                cv2.waitKey(1)
            except Exception as e:
                print("Error decoding image:", e)

            # Clear buffer
            buffer = bytearray()

            # Send the length of the received image data back to the client
            conn.send(str(len(nparr)).encode())

            print("Length sent. Length:", str(len(nparr)))

            # Wait for 1 second before processing the next image
            time.sleep(1)
        else:
            # Send a "continue sending" command
            conn.send("send more".encode())
            
    # Release resources and close the connection
    cv2.destroyAllWindows()
    conn.close()
    print("Connection closed")

Waiting for a connection on :5000
Connected to ('10.5.189.232', 61601)
Tour : 1
Expected package size: 1239126
Expected image size: 2526114
Remaining Size :1286988
Buffer : 14592
xxxxxxxxxxxxxxxx
Tour : 2
Expected package size: 570353406
Expected image size: 742367199
Remaining Size :170774667
Buffer : 43784
xxxxxxxxxxxxxxxx
Tour : 3
Expected package size: 4253741310
Expected image size: 1395765674
Remaining Size :-3429568168


OSError: [Errno 22] Invalid argument

In [26]:
import socket

# Define the host and port for the server
HOST = 'localhost'
PORT = 5000

# Create a socket and bind it to the host and port
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    print(f"Waiting for a connection on {HOST}:{PORT}")

    # Accept a connection from a client
    conn, addr = s.accept()
    print(f"Connected to {addr}")

    # Loop to receive messages from the client
    while True:
        # Receive the message data
        data = conn.recv(1024)

        # Check if the data is empty, which indicates that the client has closed the connection
        if not data:
            print("Connection closed by client")
            break

        # Decode the message data and print the message
        message = data.decode()
        print(f"Received message: {message}")

    # Close the connection
    conn.close()
    print("Connection closed")

Waiting for a connection on localhost:5000
Connected to ('127.0.0.1', 49803)
Received message: hello world
Connection closed by client
Connection closed
